## Rating csv


In [1]:
import pandas as pd
import datetime
import numpy as  np
from tqdm import tqdm
np.random.seed(1337)
with open('./kaggle/rating_train.csv', 'r') as f:
    ls = f.readlines()[1:]
u_map = {}

dates = []
foods = []
users = []
    

with tqdm(total=len(ls)) as pbar:
    for l in ls:
        date_str, user, food = l.strip().split(',')
        date = datetime.datetime.strptime(date_str, '%Y-%m-%d')
        user, food = int(user), int(food)
        if user not in u_map:
            u_map[user] = []
        u_map[user].append( (date, food) )
        
        dates.append(date)
        foods.append(food)
        users.append(user)
        pbar.update(1)
        

        
food_map = {f:i for i, f in enumerate(set(foods))}
user_map = {u:i for i, u in enumerate(set(users))}



100%|██████████| 2681494/2681494 [00:20<00:00, 131672.95it/s]


### Generate training data and label

In [57]:
### ALS embeddingimport numpy as  np
from scipy.sparse import csr_matrix
from scipy.spatial.distance import cosine
with tqdm(total=len(u_map)) as pbar:
    
#     split 1/10 for future
    eaten_map = {}
    pairs = []
    for user,v in u_map.items():
        history = v
        y = np.zeros([1, len(food_map)])
#         calculate the Y
        for d,f in history:
            y[0, food_map[f]] = 1
            pair = (user_map[user], food_map[f])
            pairs.append(pair)
        eaten_map[user] = y
        pbar.update(1)
    rows = []
    cols = []
    for pair in pairs:
        u, f = pair
        rows.append(u)
        cols.append(f)
    data = np.ones([len(rows),])
    RR = csr_matrix((data, (rows,cols)), shape=(len(user_map),len(food_map)))
    rows = []
    cols = []
    pairs = set(pairs)
    for pair in pairs:
        u, f = pair
        rows.append(u)
        cols.append(f)
    data = np.ones([len(rows),])
    R = csr_matrix((data, (rows,cols)), shape=(len(user_map),len(food_map)))
print R.shape
print 'Done'


100%|██████████| 2608/2608 [00:02<00:00, 1216.36it/s]

(2608, 5532)
Done


### SVD

In [75]:
import numpy as np
from numpy import linalg as la
a,b,c = la.svd(R.todense())
print a.shape, b.shape,c.shape

 (2608, 2608) (2608,) (5532, 5532)


In [85]:
#           


print a.shape, b.shape,c.shape
u = csr_matrix((b, (np.arange(b.shape[0]), np.arange(b.shape[0]))), shape=(a.shape[0],c.shape[0])).todense()
# 
emb = np.matmul(u,c).swapaxes(0,1)
emb = np.asarray(emb)
print emb.shape


rev_food_map = {v:k for k,v in food_map.items()}
with tqdm(total=len(u_map)) as pbar:
    with open('./SVD_predict.csv', 'w') as f:
        f.write('userid,foodid\n')
        for user in user_map.keys():
            most_eat = np.argmax(RR[user_map[user],:])
            most_food_emb = emb[most_eat,:].flatten()
#             distance = [cosine(most_food_emb, model.item_factors[i,:]) for i in range(len(food_map))]
            distance = np.matmul(emb, np.expand_dims(most_food_emb, axis=-1)).flatten()
            norms = np.linalg.norm(emb, axis=-1)
            distance = -distance/ norms
            
            recommendations = [i for i in np.argsort(distance)]
#             recommendations = [i for i,_ in model.recommend(user_map[user], user_items, N=len(food_map))]
            s = ''
            count = 0
            for food_idx in recommendations:
                if count == 20:
                    break
                if R[user_map[user], food_idx] != 1:
#                 if eaten_map[user][0, idx] != 1:
                    s += ' %d' % rev_food_map[food_idx]
                    count+=1
            f.write('%d,%s\n' % (user,s) )
            pbar.update(1)
print 'Done'

(2608, 2608) (2608,)

AttributeError: 'int' object has no attribute 'shape'

In [81]:
print distance.shape

(1, 5532)


In [50]:
d_s = 2608

b = src_b
print a.shape, b.shape,c.shape
u = csr_matrix((b, (np.arange(b.shape[0]), np.arange(b.shape[0]))), shape=(a.shape[0],c.shape[0])).todense()
p1 = np.matmul(a,u)
p2 = np.matmul(p1,c)
print p2.shape
k = 20
rev_food_map = {v:k for k,v in food_map.items()}
rev = {v:k for k,v in user_map.items()}
with open('./SVD_predict.csv', 'w') as f:
    f.write('userid,foodid\n')
    for u in range(a.shape[0]):
        arr = np.asarray(p2[u,:]).flatten()
        rank_idx = [i for i in reversed(np.argsort(arr)[-k:])]
        s = ''
        for food_idx in rank_idx:
            s += ' %d' % rev_food_map[food_idx]
        user = rev[u]
        f.write('%d,%s\n' % (user,s) )
print 'Done'
# for u in range()


(2608, 2608) (2608,) (5532, 5532)
(2608, 5532)
Done


### ALS embedding Training

In [34]:
import implicit
from implicit.als import AlternatingLeastSquares
from implicit.approximate_als import (AnnoyAlternatingLeastSquares, FaissAlternatingLeastSquares,
                                      NMSLibAlternatingLeastSquares)
from implicit.bpr import BayesianPersonalizedRanking
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,
                                         TFIDFRecommender, bm25_weight)
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'
# initialize a model
MODELS = {"als":  AlternatingLeastSquares,
#           "nmslib_als": NMSLibAlternatingLeastSquares,
          "annoy_als": AnnoyAlternatingLeastSquares,
#           "faiss_als": FaissAlternatingLeastSquares,
#           "tfidf": TFIDFRecommender,
#           "cosine": CosineRecommender,
          "bpr": BayesianPersonalizedRanking,
#           "bm25": BM25Recommender
         }
user_items = R.transpose()
    
rev = {v:k for k,v in user_map.items()}
F = [32*2**(i)for i in range(6)]
iters = [10+100*(i) for i in range(3)]

params = {'use_gpu':True}
T = prettytable.PrettyTable()
T.field_names = ["model name", "factor", "iter", "hit rate"]
best = 0.
for s in MODELS:
    for factor in F:
        for it in iters:
            model_class = MODELS.get(s)
            if s == "bpr":
                params = {'use_gpu':True, 'verify_negative_samples':True}
            else:
                params = {'use_gpu':True}

            params['factors'] = factor
            params['iterations'] = it
            
            model = model_class(**params)
            model.fit(user_items, show_progress=False)
            hits = []
            kkk=20
            idxs = np.random.permutation(len(user_map))
            for idx in idxs[:500]:
                recommendations = model.recommend(idx, user_items, N=kkk)
                hit = [Y_map[rev[idx]][0,i] != 0 for i,_ in recommendations]
                hits.append(np.mean(hit))
                # hit = [arr[i]==1 and Y_map[rev[i]][0,i] == 1for i,_ in recommendations]
            T.add_row([s, factor, it, np.mean(hits)])
            if np.mean(hits) >  best:
                best = np.mean(hits)
                attr = [s, factor, it, np.mean(hits)]
                best_model = model
print attr
print T

['als', 32, 110, 0.3795]
+------------+--------+------+----------------------+
| model name | factor | iter |       hit rate       |
+------------+--------+------+----------------------+
|    bpr     |   32   |  10  |        0.355         |
|    bpr     |   32   | 110  |        0.3217        |
|    bpr     |   32   | 210  |        0.2676        |
|    bpr     |   64   |  10  | 0.35229999999999995  |
|    bpr     |   64   | 110  | 0.32130000000000003  |
|    bpr     |   64   | 210  | 0.28790000000000004  |
|    bpr     |  128   |  10  | 0.36060000000000003  |
|    bpr     |  128   | 110  | 0.32130000000000003  |
|    bpr     |  128   | 210  | 0.30569999999999997  |
|    bpr     |  256   |  10  | 0.34890000000000004  |
|    bpr     |  256   | 110  |        0.3164        |
|    bpr     |  256   | 210  | 0.30410000000000004  |
|    bpr     |  512   |  10  |        0.3415        |
|    bpr     |  512   | 110  | 0.32820000000000005  |
|    bpr     |  512   | 210  | 0.30619999999999997  |
|  

### ALT testing with best validation model


In [84]:
### ALS embeddingimport numpy as  np
from scipy.sparse import csr_matrix
from scipy.spatial.distance import cosine
with tqdm(total=len(u_map)) as pbar:
    
#     split 1/10 for future
    eaten_map = {}
    pairs = []
    for user,v in u_map.items():
        history = v
        y = np.zeros([1, len(food_map)])
#         calculate the Y
        for d,f in history:
            y[0, food_map[f]] = 1
            pair = (user_map[user], food_map[f])
            pairs.append(pair)
        eaten_map[user] = y
        pbar.update(1)
    rows = []
    cols = []
    for pair in pairs:
        u, f = pair
        rows.append(u)
        cols.append(f)
    data = np.ones([len(rows),])
    RR = csr_matrix((data, (rows,cols)), shape=(len(user_map),len(food_map)))
    rows = []
    cols = []
    pairs = set(pairs)
    for pair in pairs:
        u, f = pair
        rows.append(u)
        cols.append(f)
    data = np.ones([len(rows),])
    R = csr_matrix((data, (rows,cols)), shape=(len(user_map),len(food_map)))
print R.shape
print 'Done'
user_items = R.transpose()
model = best_model
model.fit(user_items, show_progress=True)
            
rev_food_map = {v:k for k,v in food_map.items()}
with tqdm(total=len(u_map)) as pbar:
    with open('./ALS_predict.csv', 'w') as f:
        f.write('userid,foodid\n')
        for user in user_map.keys():
#             most_eat = np.argmax(RR[user_map[user],:])
#             most_food_emb = model.item_factors[most_eat,:]
#             distance = [cosine(most_food_emb, model.item_factors[i,:]) for i in range(len(food_map))]
#             distance = np.matmul(model.item_factors, np.expand_dims(most_food_emb,axis=-1)).flatten()
#             norms = np.linalg.norm(model.item_factors, axis=-1)
#             distance = -distance/ norms
            
#             recommendations = [i for i in np.argsort(distance)]
            recommendations = [i for i,_ in model.recommend(user_map[user], user_items, N=len(food_map))]
            s = ''
            c = 0
            for food_idx in recommendations:
                if c == 20:
                    break
                y = eaten_map[user]
                if R[user_map[user], food_idx] == 0:
                    s += ' %d' % rev_food_map[food_idx]
                    c+=1
            f.write('%d,%s\n' % (user,s) )
            pbar.update(1)
print 'Done'
# for u in range()


100%|██████████| 2608/2608 [00:02<00:00, 1236.52it/s]
 13%|█▎        | 14.5/110 [00:00<00:00, 144.49it/s]

(2608, 5532)
Done


100%|██████████| 110.0/110 [00:00<00:00, 196.67it/s]
100%|██████████| 2608/2608 [00:27<00:00, 94.32it/s]

Done


In [50]:
with tqdm(total=len(u_map)) as pbar:
    with open('./ALS_predict.csv', 'w') as f:
        f.write('userid,foodid\n')
        for user in user_map.keys():
            most_eat = np.argmax(RR[user_map[user],:])
            most_food_emb = model.item_factors[most_eat,:]
            distance = np.matmul(model.item_factors, np.expand_dims(most_food_emb,axis=-1)).flatten()
            recommendations = [i for i in np.argsort(distance)]
#             recommendations = [i for i,_ in model.recommend(user_map[user], user_items, N=len(food_map))]
            s = ''
            c = 0
            for food_idx in recommendations:
                if c == 20:
                    break
                y = eaten_map[user]
                if y[0, idx] != 1:
#                 if eaten_map[user][0, idx] != 1:
                    s += ' %d' % rev_food_map[food_idx]
                    c+=1
            f.write('%d,%s\n' % (user,s) )
            pbar.update(1)
print 'Done'
# for u in range()

100%|██████████| 2608/2608 [00:03<00:00, 718.41it/s]

Done


In [52]:
# model.item_factors.shape
print user
most_eat = np.argmax(RR[user_map[user],:])
print RR[user_map[user],most_eat]
# most_food_emb = model.item_factors[most_eat,:]
# distance = np.matmul(model.item_factors, np.expand_dims(most_food_emb,axis=-1))


8191
42.0


### ALS backup

In [ ]:
import implicit
from implicit.als import AlternatingLeastSquares
from implicit.approximate_als import (AnnoyAlternatingLeastSquares, FaissAlternatingLeastSquares,
                                      NMSLibAlternatingLeastSquares)
from implicit.bpr import BayesianPersonalizedRanking
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,
                                         TFIDFRecommender, bm25_weight)
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'
# initialize a model
MODELS = {"als":  AlternatingLeastSquares,
#           "nmslib_als": NMSLibAlternatingLeastSquares,
          "annoy_als": AnnoyAlternatingLeastSquares,
#           "faiss_als": FaissAlternatingLeastSquares,
#           "tfidf": TFIDFRecommender,
#           "cosine": CosineRecommender,
          "bpr": BayesianPersonalizedRanking,
#           "bm25": BM25Recommender
         }


def get_model(model_name):
    print("getting model %s" % model_name)
    model_class = MODELS.get(model_name)
    if not model_class:
        raise ValueError("Unknown Model '%s'" % model_name)

    # some default params
    if issubclass(model_class, AlternatingLeastSquares):
        params = {'factors': 64, 'dtype': np.float32, 'iterations':100}
    elif model_name == "bpr":
        params = {'factors': 512, 'dtype': np.float32, 'iterations':300, 'verify_negative_samples':True}
    elif model_name == "bm25":
        params = {'K1': 100, 'B': 0.5}
    if model_name == 'faiss_als':
        params['use_gpu'] = False
    return model_class(**params)
# model = implicit.als.AlternatingLeastSquares(factors=256, use_gpu=True, iterations=1000)
rev = {v:k for k,v in user_map.items()}

for s in MODELS:
    model = get_model(s)
    user_items = R.transpose()
    model.fit(user_items, show_progress=True)
    hits = []
    kkk=20
    for idx in range(12,64):
        recommendations = model.recommend(idx, user_items, N=kkk)
        hit = [Y_map[rev[idx]][0,i] != 0 for i,_ in recommendations]
        hits.append(np.mean(hit))
        # hit = [arr[i]==1 and Y_map[rev[i]][0,i] == 1for i,_ in recommendations]
    print s, np.mean(hits)### ALS embedding Training